In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, LSTM

In [2]:
nan_fill = -160
train_size = 0.7
timesteps=5 # next time dbm Prediction is based on last how many past time dbm

In [3]:
def get_table_dbm(df, max_id=50):
    tmp_df=df.sort_values(by='id')
    t_tx = pd.DataFrame(columns=['time']+['id_'+str(i) for i in range(1, max_id+1)])
    for time_step in tmp_df['time'].unique():
        t = tmp_df.loc[tmp_df['time']==time_step, ['id','dbm']].T
        t.columns = ['id_'+str(int(i)) for i in t.loc['id'].tolist()]
        t['time'] = time_step
        t = t.drop(index='id')
        t_tx = pd.concat([t_tx, t])
    t_tx = t_tx.sort_values(by='time').reset_index(drop=True)
    return t_tx

In [4]:
df = pd.read_csv('50sensor_power_2tx_latlon_5g.csv')
df.head()

,time,tx,id,power,lon,lat,dbm
0,0.00,500,16,4.996616e-13,3.06045,50.63294,-123.0
1,0.02,500,16,5.004682e-13,3.06045,50.63294,-123.0
2,0.04,500,16,5.013803e-13,3.06045,50.63294,-123.0
3,0.06,500,16,5.021966e-13,3.06045,50.63294,-123.0
4,0.08,500,16,5.031786e-13,3.06045,50.63294,-123.0


In [5]:
data = get_table_dbm(df)
data.head()

,time,id_1,id_2,id_3,id_4,id_5,id_6,id_7,id_8,id_9,...,id_41,id_42,id_43,id_44,id_45,id_46,id_47,id_48,id_49,id_50
0,0.00,NaN,-98.0,NaN,NaN,NaN,NaN,-97.0,-128.0,NaN,...,-115.0,-97.0,-107.0,-60.0,-80.0,NaN,-127.0,-102.0,NaN,NaN
1,0.02,NaN,-98.0,NaN,NaN,NaN,NaN,-97.0,-128.0,NaN,...,-114.0,-98.0,-108.0,-61.0,-102.0,NaN,-127.0,-101.0,NaN,NaN
2,0.04,NaN,-95.0,NaN,NaN,NaN,NaN,-97.0,-128.0,NaN,...,-113.0,-95.0,-108.0,-60.0,-105.0,NaN,-127.0,-102.0,NaN,NaN
3,0.06,NaN,-94.0,NaN,NaN,NaN,NaN,-97.0,-128.0,NaN,...,-112.0,-99.0,-109.0,-61.0,-104.0,NaN,-127.0,-104.0,NaN,NaN
4,0.08,NaN,-95.0,NaN,NaN,NaN,NaN,-97.0,-128.0,NaN,...,-112.0,-110.0,-109.0,-62.0,-103.0,NaN,-127.0,-101.0,NaN,NaN


In [6]:
data = data.fillna(nan_fill)
data.head()

,time,id_1,id_2,id_3,id_4,id_5,id_6,id_7,id_8,id_9,...,id_41,id_42,id_43,id_44,id_45,id_46,id_47,id_48,id_49,id_50
0,0.00,-160,-98.0,-160,-160.0,-160.0,-160.0,-97.0,-128.0,-160.0,...,-115.0,-97.0,-107.0,-60.0,-80.0,-160,-127.0,-102.0,-160.0,-160.0
1,0.02,-160,-98.0,-160,-160.0,-160.0,-160.0,-97.0,-128.0,-160.0,...,-114.0,-98.0,-108.0,-61.0,-102.0,-160,-127.0,-101.0,-160.0,-160.0
2,0.04,-160,-95.0,-160,-160.0,-160.0,-160.0,-97.0,-128.0,-160.0,...,-113.0,-95.0,-108.0,-60.0,-105.0,-160,-127.0,-102.0,-160.0,-160.0
3,0.06,-160,-94.0,-160,-160.0,-160.0,-160.0,-97.0,-128.0,-160.0,...,-112.0,-99.0,-109.0,-61.0,-104.0,-160,-127.0,-104.0,-160.0,-160.0
4,0.08,-160,-95.0,-160,-160.0,-160.0,-160.0,-97.0,-128.0,-160.0,...,-112.0,-110.0,-109.0,-62.0,-103.0,-160,-127.0,-101.0,-160.0,-160.0


In [7]:
data.shape

(3001, 51)

In [8]:
data['time'].max()

60.0

## Preparing the data for LSTM

In [9]:
length = data.shape[0]
length

3001

In [10]:
data_fea = data.iloc[:, 1 :]
data_fea.head()

,id_1,id_2,id_3,id_4,id_5,id_6,id_7,id_8,id_9,id_10,...,id_41,id_42,id_43,id_44,id_45,id_46,id_47,id_48,id_49,id_50
0,-160,-98.0,-160,-160.0,-160.0,-160.0,-97.0,-128.0,-160.0,-116.0,...,-115.0,-97.0,-107.0,-60.0,-80.0,-160,-127.0,-102.0,-160.0,-160.0
1,-160,-98.0,-160,-160.0,-160.0,-160.0,-97.0,-128.0,-160.0,-119.0,...,-114.0,-98.0,-108.0,-61.0,-102.0,-160,-127.0,-101.0,-160.0,-160.0
2,-160,-95.0,-160,-160.0,-160.0,-160.0,-97.0,-128.0,-160.0,-118.0,...,-113.0,-95.0,-108.0,-60.0,-105.0,-160,-127.0,-102.0,-160.0,-160.0
3,-160,-94.0,-160,-160.0,-160.0,-160.0,-97.0,-128.0,-160.0,-115.0,...,-112.0,-99.0,-109.0,-61.0,-104.0,-160,-127.0,-104.0,-160.0,-160.0
4,-160,-95.0,-160,-160.0,-160.0,-160.0,-97.0,-128.0,-160.0,-117.0,...,-112.0,-110.0,-109.0,-62.0,-103.0,-160,-127.0,-101.0,-160.0,-160.0


In [11]:
data_fea = data_fea.to_numpy()
data_fea[: 3]

array([[-160.,  -98., -160., -160., -160., -160.,  -97., -128., -160.,
        -116., -160., -113., -160.,  -85.,  -78., -123.,  -98.,  -82.,
        -112.,  -98.,  -97., -112.,  -93., -160., -118., -102., -105.,
         -55., -111., -160., -110.,  -91., -108.,  -73.,  -94., -114.,
        -110.,  -75., -112., -113., -115.,  -97., -107.,  -60.,  -80.,
        -160., -127., -102., -160., -160.],
       [-160.,  -98., -160., -160., -160., -160.,  -97., -128., -160.,
        -119., -160., -114., -160.,  -72.,  -76., -123.,  -97.,  -81.,
        -114.,  -99.,  -96., -112.,  -92., -160., -119., -101., -105.,
         -55., -111., -166., -108.,  -89., -106.,  -74.,  -95., -118.,
        -110.,  -78., -104., -113., -114.,  -98., -108.,  -61., -102.,
        -160., -127., -101., -160., -160.],
       [-160.,  -95., -160., -160., -160., -160.,  -97., -128., -160.,
        -118., -160., -114., -160.,  -72.,  -77., -123.,  -97.,  -81.,
        -113.,  -96.,  -96., -111.,  -93., -160., -119.,  -9

In [12]:
# min max scaler
sc=MinMaxScaler()

DataScaler = sc.fit(data_fea)
data_fea=DataScaler.transform(data_fea)
data_fea[: 3]

array([[0.        , 0.45070423, 0.        , 0.        , 0.        ,
        0.        , 0.64948454, 0.46376812, 0.        , 0.39285714,
        0.        , 0.78333333, 0.        , 0.67123288, 0.82727273,
        0.578125  , 0.73202614, 0.61904762, 0.38571429, 0.39473684,
        0.57272727, 0.64      , 0.4       , 0.        , 0.54545455,
        0.36      , 0.57798165, 1.        , 0.42608696, 0.07792208,
        0.66666667, 0.525     , 0.26760563, 0.725     , 0.69473684,
        0.23655914, 0.87719298, 0.828125  , 0.48      , 0.55294118,
        0.66037736, 0.4       , 0.56382979, 0.89285714, 0.7079646 ,
        0.        , 0.12820513, 0.59793814, 0.        , 0.        ],
       [0.        , 0.45070423, 0.        , 0.        , 0.        ,
        0.        , 0.64948454, 0.46376812, 0.        , 0.36607143,
        0.        , 0.76666667, 0.        , 0.84931507, 0.84545455,
        0.578125  , 0.73856209, 0.63095238, 0.35714286, 0.38157895,
        0.58181818, 0.64      , 0.41818182, 0. 

In [13]:
X_data = list()
y_data = list()

# Iterate thru the values to create combinations
for i in range(timesteps , length , 1):
    x_sample = data_fea[i-timesteps:i]
    y_sample = data_fea[i]
    X_data.append(x_sample)
    y_data.append(y_sample)

In [14]:
X_data[0].shape

(5, 50)

In [15]:
X_data = np.array(X_data)
y_data = np.array(y_data)

print('X_data shape: ',X_data.shape)
print('y_data shape: ',y_data.shape)

X_data shape:  (2996, 5, 50)
y_data shape:  (2996, 50)


## split train and text

In [16]:
train_x = X_data[: int(length * train_size)]
train_y = y_data[: int(length * train_size)]

test_x = X_data[int(length * train_size) :]
test_y = y_data[int(length * train_size) :]

print('train_x shape:',train_x.shape)
print('train_y shape:',train_y.shape)

print('test_x shape:',test_x.shape)
print('test_y shape:',test_y.shape)

train_x shape: (2100, 5, 50)
train_y shape: (2100, 50)
test_x shape: (896, 5, 50)
test_y shape: (896, 50)


In [17]:
timesteps

5

In [18]:
totalfeatures = train_x.shape[2]
totalfeatures

50

In [19]:
train_x[1]

array([[0.        , 0.45070423, 0.        , 0.        , 0.        ,
        0.        , 0.64948454, 0.46376812, 0.        , 0.36607143,
        0.        , 0.76666667, 0.        , 0.84931507, 0.84545455,
        0.578125  , 0.73856209, 0.63095238, 0.35714286, 0.38157895,
        0.58181818, 0.64      , 0.41818182, 0.        , 0.53246753,
        0.37333333, 0.57798165, 1.        , 0.42608696, 0.        ,
        0.69333333, 0.55      , 0.29577465, 0.71666667, 0.68421053,
        0.19354839, 0.87719298, 0.8046875 , 0.56      , 0.55294118,
        0.67924528, 0.38333333, 0.55319149, 0.88392857, 0.51327434,
        0.        , 0.12820513, 0.60824742, 0.        , 0.        ],
       [0.        , 0.49295775, 0.        , 0.        , 0.        ,
        0.        , 0.64948454, 0.46376812, 0.        , 0.375     ,
        0.        , 0.76666667, 0.        , 0.84931507, 0.83636364,
        0.578125  , 0.73856209, 0.63095238, 0.37142857, 0.42105263,
        0.58181818, 0.65333333, 0.4       , 0. 

In [20]:
train_y[1]

array([0.        , 0.42253521, 0.        , 0.        , 0.        ,
       0.        , 0.59793814, 0.47826087, 0.        , 0.40178571,
       0.        , 0.76666667, 0.        , 0.78082192, 0.8       ,
       0.578125  , 0.73202614, 0.4047619 , 0.21428571, 0.47368421,
       0.59090909, 0.65333333, 0.45454545, 0.        , 0.58441558,
       0.33333333, 0.60550459, 0.98550725, 0.43478261, 0.        ,
       0.76      , 0.5625    , 0.3943662 , 0.74166667, 0.66315789,
       0.21505376, 0.85964912, 0.8671875 , 0.54      , 0.49411765,
       0.71698113, 0.48333333, 0.53191489, 0.91964286, 0.49557522,
       0.        , 0.12820513, 0.6185567 , 0.        , 0.        ])

In [28]:
lr = 0.001
epoch = 50
batchsize = 5

In [22]:
regressor = Sequential()
 
# Adding the First input hidden layer and the LSTM layer
# return_sequences = True, means the output of every time step to be shared with hidden next layer
regressor.add(LSTM(units = 64, activation = 'relu', input_shape = (timesteps, totalfeatures), return_sequences=True))
 
# Adding the Second Second hidden layer and the LSTM layer
regressor.add(LSTM(units = 32, activation = 'relu', input_shape = (timesteps, totalfeatures), return_sequences=True))
 
# Adding the Second Third hidden layer and the LSTM layer
regressor.add(LSTM(units = 16, activation = 'relu', return_sequences=False ))
 
# Adding the output layer
regressor.add(Dense(units = totalfeatures))
 
# Compiling the LSTM
regressor.compile(optimizer = optimizers.Adam(learning_rate=lr), loss = 'mean_squared_error', metrics=['mae'])
 
##################################################
 
import time
# Measuring the time taken by the model to train
StartTime=time.time()
 
# Fitting the LSTM to the Training set
regressor.fit(train_x, train_y, batch_size = batchsize, epochs = epoch)
 
EndTime=time.time()
print("## Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ##')

Epoch 1/50
420/420 [==============================] - 6s 7ms/step - loss: 0.0475 - mae: 0.1495
Epoch 2/50
420/420 [==============================] - 3s 7ms/step - loss: 0.0175 - mae: 0.0910
Epoch 3/50
420/420 [==============================] - 4s 8ms/step - loss: 0.0129 - mae: 0.0772
Epoch 4/50
420/420 [==============================] - 3s 8ms/step - loss: 0.0110 - mae: 0.0708
Epoch 5/50
420/420 [==============================] - 3s 7ms/step - loss: 0.0098 - mae: 0.0666
Epoch 6/50
420/420 [==============================] - 3s 7ms/step - loss: 0.0092 - mae: 0.0646
Epoch 7/50
420/420 [==============================] - 3s 8ms/step - loss: 0.0088 - mae: 0.0629
Epoch 8/50
420/420 [==============================] - 3s 8ms/step - loss: 0.0084 - mae: 0.0614
Epoch 9/50
420/420 [==============================] - 3s 8ms/step - loss: 0.0081 - mae: 0.0602
Epoch 10/50
420/420 [==============================] - 3s 8ms/step - loss: 0.0078 - mae: 0.0585
Epoch 11/50
420/420 [============================

In [23]:
regressor.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 5, 64)             29440     
                                                                 
 lstm_2 (LSTM)               (None, 5, 32)             12416     
                                                                 
 lstm_3 (LSTM)               (None, 16)                3136      
                                                                 
 dense (Dense)               (None, 50)                850       
                                                                 
Total params: 45,842
Trainable params: 45,842
Non-trainable params: 0
_________________________________________________________________


## Measuring the accuracy of the model on testing data

In [24]:
# Making predictions on test data
predicted_dbm = regressor.predict(test_x)
predicted_dbm = DataScaler.inverse_transform(predicted_dbm)
predicted_dbm = predicted_dbm.astype(int)

 # Getting the original dbm values for testing data
orig=test_y
orig=DataScaler.inverse_transform(test_y)
 
# Accuracy of the predictions
print('Accuracy:', 100 - (100*(abs(orig-predicted_dbm)/orig)).mean())

Accuracy: 111.6300413066446


In [25]:
mae = mean_absolute_error(y_true=orig, y_pred=predicted_dbm)
mae

12.565111607142855

In [26]:
mse = mean_squared_error(y_true=orig, y_pred=predicted_dbm)
mse

333.34533482142865

In [27]:
print('id'+'\t'+'real dbm'+'\t\t'+'predict')
for i in range(orig[0].shape[0]):
    print(i+1,'\t', orig[0][i],'\t\t', predicted_dbm[0][i])

id	real dbm		predict
1 	 -160.0 		 -160
2 	 -89.0 		 -112
3 	 -160.0 		 -160
4 	 -160.0 		 -157
5 	 -160.0 		 -165
6 	 -160.0 		 -152
7 	 -160.0 		 -151
8 	 -160.0 		 -159
9 	 -111.0 		 -116
10 	 -81.0 		 -97
11 	 -160.0 		 -155
12 	 -160.0 		 -154
13 	 -160.0 		 -158
14 	 -121.0 		 -109
15 	 -116.0 		 -109
16 	 -160.0 		 -158
17 	 -128.0 		 -122
18 	 -110.0 		 -108
19 	 -126.0 		 -114
20 	 -61.0 		 -64
21 	 -115.0 		 -115
22 	 -117.0 		 -116
23 	 -104.0 		 -102
24 	 -160.0 		 -160
25 	 -115.0 		 -94
26 	 -101.0 		 -104
27 	 -75.0 		 -123
28 	 -102.00000000000001 		 -100
29 	 -59.0 		 -71
30 	 -160.0 		 -168
31 	 -100.0 		 -100
32 	 -100.0 		 -94
33 	 -112.0 		 -108
34 	 -103.0 		 -107
35 	 -97.00000000000001 		 -106
36 	 -78.0 		 -69
37 	 -116.00000000000001 		 -120
38 	 -115.0 		 -122
39 	 -160.0 		 -151
40 	 -160.0 		 -161
41 	 -116.0 		 -120
42 	 -96.0 		 -100
43 	 -86.0 		 -94
44 	 -91.0 		 -102
45 	 -95.0 		 -95
46 	 -160.0 		 -160
47 	 -104.0 		 -100
48 	 -84.0 		 -89
49 	 -119.